# 第一节 加载公开词向量

In [33]:
import jieba
# 可以把解压后的文件导入，也可以把压缩包导入，支持自动解压
from gensim.models import KeyedVectors

model_path = '../../Data/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5.bz2'
model = KeyedVectors.load_word2vec_format(model_path)

KeyboardInterrupt: 

In [9]:
print(model.vector_size)
print(model['地铁'])

300
[ 0.567997  0.158654  0.038625  0.249258 -0.196332  0.213017 -0.136925
  0.232799  0.144968 -0.09523   0.425262 -0.092607  0.694388  0.025083
 -0.183769 -0.201074  0.223489 -0.26399  -0.072578  0.147112  0.353904
 -0.166529 -0.194957 -0.026473 -0.113792  0.185063  0.357284  0.542023
  0.231774  0.57543  -0.405007 -0.163268  0.269997 -0.120403  0.414961
 -0.066652 -0.421365 -0.015576 -0.012792 -0.184059 -0.110147 -0.007924
  0.036816 -0.154306  0.230923 -0.083318 -0.516196 -0.538834  0.117642
 -0.210153 -0.124584 -0.212592 -0.05337   0.329235 -0.076741 -0.467959
 -0.294227 -0.00829  -0.243343  0.056438  0.028466  0.117502  0.62926
 -0.650896 -0.295166 -0.244159  0.421573  0.115636  0.212001  0.418772
 -0.40834  -0.265115 -0.042153  0.055105 -0.323379 -0.493779  0.02188
  0.180165 -0.222311  0.024993 -0.417476 -0.060329  0.050551 -0.329032
 -0.675789 -0.372138 -0.246428  0.265466 -0.1353    0.188153 -0.054184
 -0.288865  0.596618 -0.058765 -0.349856  0.044062 -0.042164 -0.238806
  0.

In [4]:
similarity = model.similarity('地铁', '公交')
print('地铁 vs 公交 相似度：', similarity)

地铁 vs 公交 相似度： 0.5874173


In [5]:
similar_words = model.most_similar(positive=["上班"], topn=5)
print(similar_words)
result = model.most_similar(positive=["爸爸", "女性"], negative=["男性"], topn=3)
print(result)

[('下班', 0.7061899304389954), ('上夜班', 0.644236147403717), ('上下班', 0.6376184225082397), ('值完', 0.6258902549743652), ('照常上班', 0.6204943060874939)]
[('妈妈', 0.7957949638366699), ('母亲', 0.6855213046073914), ('爷爷', 0.6502836346626282)]


d:\CodeRelated\codeService\anaconda3\envs\npl-learning\Lib\site-packages\gensim\models\keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


# 第二节 训练自己的词向量

In [48]:
import pandas as pd
import jieba
from gensim.models import Word2Vec, KeyedVectors

In [49]:
df = pd.read_csv('../../Data/online_shopping_10_cats.csv', encoding='utf-8', sep=',').dropna()  # sep默认是逗号

In [50]:
df.head

<bound method NDFrame.head of       cat  label                                             review
0      书籍      1  做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1      书籍      1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2      书籍      1  作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3      书籍      1  作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4      书籍      1  作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...
...    ..    ...                                                ...
62769  酒店      0  我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...
62770  酒店      0  房间很小，整体设施老化，和四星的差距很大。毛巾太破旧了。早餐很简陋。房间隔音很差，隔两间房间...
62771  酒店      0                      我感觉不行。。。性价比很差。不知道是银川都这样还是怎么的！
62772  酒店      0  房间时间长，进去有点异味！服务员是不是不够用啊！我在一楼找了半个小时以上才找到自己房间，想找...
62773  酒店      0  老人小孩一大家族聚会，选在吴宫泛太平洋，以为新加坡品牌一定很不错，没想到11点30分到前台，...

[62773 rows x 3 columns]>

In [51]:
# df['review']
sentences = [[token for token in jieba.lcut(sentence) if token.strip() != ''] for sentence in df['review']]

In [52]:
sentences[0:3]

[['做',
  '父母',
  '一定',
  '要',
  '有',
  '刘墉',
  '这样',
  '的',
  '心态',
  '，',
  '不断',
  '地',
  '学习',
  '，',
  '不断',
  '地',
  '进步',
  '，',
  '不断',
  '地',
  '给',
  '自己',
  '补充',
  '新鲜血液',
  '，',
  '让',
  '自己',
  '保持',
  '一颗',
  '年轻',
  '的',
  '心',
  '。',
  '我',
  '想',
  '，',
  '这',
  '是',
  '他',
  '能',
  '很',
  '好',
  '的',
  '和',
  '孩子',
  '沟通',
  '的',
  '一个',
  '重要',
  '因素',
  '。',
  '读',
  '刘墉',
  '的',
  '文章',
  '，',
  '总能',
  '让',
  '我',
  '看到',
  '一个',
  '快乐',
  '的',
  '平易近人',
  '的',
  '父亲',
  '，',
  '他',
  '始终',
  '站',
  '在',
  '和',
  '孩子',
  '同样',
  '的',
  '高度',
  '，',
  '给',
  '孩子',
  '创造',
  '着',
  '一个',
  '充满',
  '爱',
  '和',
  '自由',
  '的',
  '生活',
  '环境',
  '。',
  '很',
  '喜欢',
  '刘墉',
  '在',
  '字里行间',
  '流露出',
  '的',
  '做',
  '父母',
  '的',
  '那种',
  '小',
  '狡黠',
  '，',
  '让',
  '人',
  '总是',
  '忍俊不禁',
  '，',
  '父母',
  '和',
  '子女',
  '之间',
  '有时候',
  '也',
  '是',
  '一种',
  '战斗',
  '，',
  '武力',
  '争斗',
  '过于',
  '低级',
  '了',
  '，',
  '智力',
  '较量',
  '才',
  '更',
  '有',
  '趣味',
  '。',
  '

In [53]:
model = Word2Vec(
    sentences,  # 已分词的句子序列
    vector_size=100,  # 词向量的维度
    window=5,  # 上下文窗口大小
    min_count=2,  # 最小词频（低于将被忽略）
    sg=1,  # 1：Skip-Gram，0：CBOW
    workers=4,  # 并行训练线程数
)# 实例化模型的同时确实开始了词向量的训练。

In [54]:
model.wv['地铁']
model.wv.vector_size

100

In [55]:
model.wv.save_word2vec_format('./data/word2vec.txt')

In [56]:
KeyedVectors.load_word2vec_format('./data/word2vec.txt')['地铁']

array([ 0.22488853,  0.42092636, -0.17300309,  0.17577414,  0.28958657,
       -0.5624829 ,  0.20848933,  0.43081263, -0.6154931 ,  0.06341657,
       -0.26661837, -0.41219366,  0.59604734, -0.27236134,  0.55199015,
        0.23198873,  0.6127713 , -0.66558105, -0.2246045 , -0.3613789 ,
        0.68532574, -0.23928666,  0.36542597,  0.01410825, -0.30508533,
        0.09304506,  0.5409768 , -0.01797713, -0.18766832, -0.16107774,
       -0.08907314, -0.31648368, -0.14819802, -0.3167416 ,  0.6365136 ,
       -0.0278817 ,  0.17788051,  0.21568221,  0.17012103, -0.40443513,
       -0.23867543,  0.29978076, -0.2011098 ,  0.37098473,  0.7842728 ,
       -0.16457075, -0.05510465,  0.35331112, -0.04366172,  0.34606618,
       -0.4136292 ,  0.2780064 ,  0.07506328, -0.18518507,  0.65557855,
       -0.20213437,  0.25697568,  0.41541022, -0.474141  ,  0.34733754,
       -0.22313502,  0.3987913 ,  0.1992021 , -0.07571858, -0.09558831,
       -0.04716373,  0.46716234,  0.70260066, -0.03138976,  0.32

# 第三节 词向量应用

#### 区分key_to_index和index_to_key返回数据的类型
```pthon
# 训练一个简单的 Word2Vec 模型
model = Word2Vec(sentences, vector_size=10, window=5, min_count=1, sg=0)

# key_to_index: 单词到索引的映射
print("Key to Index:", model.wv.key_to_index)
# 输出: Key to Index: {'apple': 0, 'banana': 1, 'cherry': 2, 'dog': 3, 'cat': 4}

# index_to_key: 索引到单词的映射
print("Index to Key:", model.wv.index_to_key)
# 输出: Index to Key: ['apple', 'banana', 'cherry', 'dog', 'cat']
```

In [34]:
from torch import nn
from gensim.models import KeyedVectors
import torch
import jieba

In [35]:
# 1.准备好词向量矩阵
wv = KeyedVectors.load_word2vec_format('./data/word2vec.txt')

In [36]:
# 处理OOV
# 在原有的此表上加入新词
# 一般讲特殊词汇加入到开头（习惯上）
unk_token = '<unk>'
index2word = [unk_token] + wv.index_to_key
word2index = {word: index for index, word in enumerate(index2word)}

In [37]:
# 2.准备词向量矩阵
num_embeddings = len(index2word)  # 检索出词向量
print("num_embeddings", num_embeddings)
embedding_dim = wv.vector_size
print("embedding_dim", embedding_dim)
embedding_matrix = torch.randn(num_embeddings, embedding_dim)  # 创建 (num_embeddings, embedding_dim)形状的tensor张量
# print(embedding_matrix)
for index, word in enumerate(index2word):
    if word in wv:
        embedding_matrix[index] = torch.tensor(wv[word])

num_embeddings 34577
embedding_dim 100


In [42]:
# 3.创建Embedding
embedding = nn.Embedding.from_pretrained(embedding_matrix)
print(embedding)

Embedding(34577, 100)


In [46]:
# 4.测试
# 这里容易出现OOV问题
text = "给我擦皮鞋"
tokens = jieba.lcut(text)
print(tokens)
input_ids = [word2index.get(token,word2index[unk_token]) for token in tokens]
input_tensor = torch.tensor(input_ids)
print(input_tensor)
embedding(input_tensor)


['给', '我', '擦皮鞋']
tensor([28,  8,  0])


tensor([[-0.1968,  0.1588,  0.1895, -0.1914,  0.7607, -0.1859,  0.4304,  0.2327,
         -0.0793, -0.0188,  0.3871, -0.6174,  0.4740, -0.0052, -0.1786, -0.1930,
         -0.2753, -0.0126,  0.1849, -0.2702,  0.1057,  0.1456,  0.6507,  0.3690,
         -0.1901,  0.0398,  0.1246, -0.1694, -0.0733, -0.6120,  0.5980,  0.0599,
          0.2279, -0.4735, -0.4726,  0.3080, -0.1696,  0.2086, -0.1610, -0.2779,
         -0.1692, -0.3756,  0.1074,  0.3943, -0.2522, -0.1471,  0.1519, -0.4709,
          0.0083, -0.0592, -0.4171, -0.5792,  0.3001, -0.0475, -0.3524,  0.2259,
         -0.0697,  0.1007, -0.2514, -0.0028,  0.0928, -0.3455,  0.2034, -0.8628,
         -0.2193, -0.0650,  0.1839,  0.2649, -0.2523, -0.1014, -0.6896, -0.4340,
          0.5302, -0.1026, -0.0930,  0.3763,  0.0123, -0.3609, -0.4472, -0.1509,
          0.0358,  0.3734,  0.2250,  0.4212, -0.2468, -0.4958,  0.1944, -0.2363,
          0.1973, -0.1005,  0.1292, -0.3063,  0.0929, -0.2669,  1.3386,  0.1616,
          0.1939,  0.2351, -